# Catenaria

Para el problema de la catenaria, de hallar la posición de los eslabones
de una cadena que une los puntos $(X_0, Y_0)$ con $(X_f, Y_f)$, con eslabones de longitud
1, que minimice la energía potencial de la misma (ver Luenberger, página 330).

### a)
> Hallar condiciones en las que el problema sea factibe, y una aproximación inicla adecuada (que podría ser hallada como punto de partida para un esquema iterativo).

<br/><br/>
Si consideramos los eslabones numerados del $0$ al $19$ y que el eslabón $i$ tiene un desplazamiento $x_i$ en $x$ y un desplazamiento $y_i$ en $y$, es suficiente que el desplazamiento total en $y$ sea cero mientras que en $x$ sea $16$. Para una aproximación inicial, imagino la cadena tensada hacia abajo por efecto por ejemplo de un peso en el medio de manera tal de que forme un triángulo. Notemos que nos quedan diez eslabones de cada lado, es decir un triángulo isóceles de lado 10, base 16 y altura 6. Está claro que esta solución no es óptima, pero resulta conveniente para escribir explícitamente ya que los desplazamientos de los eslabones son uniformes, por lo tanto nos queda:

$$x_i = \frac{16}{20}, \ 0 \leq i \leq 19 $$
$$ y_i = \frac{6}{10}, \ 0 \leq i \leq 9 $$
$$ y_i = \frac{-6}{10}, \ 10 \leq i \leq 19. $$

A continuación ploteo las uniones de los eslabones para esta configuración.

In [50]:
import numpy as np
import plotly.graph_objs as go

coord_x = np.array(
    [
        0.0,
        0.8,
        1.6,
        2.4,
        3.2,
        4.0,
        4.8,
        5.6,
        6.4,
        7.2,
        8.0,
        8.8,
        9.6,
        10.4,
        11.2,
        12.0,
        12.8,
        13.6,
        14.4,
        15.2,
        16.0,
    ]
)
coord_y = np.array(
    [
        0.0,
        0.6,
        1.2,
        1.8,
        2.4,
        3.0,
        3.6,
        4.2,
        4.8,
        5.4,
        6.0,
        5.4,
        4.8,
        4.2,
        3.6,
        3.0,
        2.4,
        1.8,
        1.2,
        0.6,
        0.0,
    ]
)
fig = go.Figure()
fig.add_trace(go.Scatter(x=coord_x, y=coord_y, mode='markers'))
fig.update_yaxes(range=[6.5, -0.5])
fig.show()

### b)
> Elegir variables para describir el problema distintas de las del libro, plantear las función objetivo y las restricciones en esas variables, u escrbir las condiciones necesarias de primer orden

<br/><br/>
Considero para cada eslabón 4 variables $x_i^l, x_i^r, y_i^l, y_i^r$, las cuales representan las coordenadas de los extremos izquierdo y derecho del eslabón $i$ (ver figura). Para ser congruente con el indexado de python, numero las coordenadas de $x$ empezando en cero, y les asigno el siguiente orden: $x = (x_0^l, y_0^l, x_0^r, y_0^r, ..., x_{19}^l, y_{19}^l, x_{19}^r, y_{19}^r).$

![Eslabón i](catenaria_eslabon_i.png)
<center/>Eslabón $i$<center/>
<br/><br/>

Comenzando por las restricciones de vínculo, tenemos que $x_0^l = y_0^l = y_{19}^r = 0$ y $x_{19}^r = 16$. Notemos además que el extremo derecho de un eslabón debe coincidir con el izquierdo del siguiente, es decir que para todo $2 \leq i \leq 20$, tenemos $x_i^l = x_{i-1}^r$ y $y_i^l = y_{i-1}^r$. Por último tenemos que los eslabones son inextensibles, es decir que $\sqrt{(x_i^r - x_i^l)^2 + (y_i^r - y_i^r)^2} = 1$. Con todo esto tenemos entonces $2*19 + 4 + 20 = 62$ restricciones. El objetivo sigue siendo minimizar la energía potencial total de la cadena, que en este caso considerando los eslabones de masa unitaria, viene dada por 

$$ \sum_{i=1}^{20} g \frac{(y_i^l + y_i^r)}{2} $$

Si igualamos todas las restricciones a 0 y las numeramos $h_1, \dots, h_{62}$, entonces según las condiciones de primer orden un punto regular $x^*$ resultará un mínimo si $\exists \lambda \in \mathbb{R}^{62}$ tal que resulte

$$ \nabla{f} + \lambda^T \mathbf{\nabla{h}} = \mathbf{0}. $$

donde $\mathbf{h}$ es un arreglo matricial de las $h_i$ una encima de la otra. A continuación defino la función objetivo de esta configuración y una función auxiliar para computar el tamaño de un eslabón dado.

In [147]:
from scipy.constants import g
def obj_function(x):
    """Función objetivo a minimizar.
    
    Parámetros
    ----------
    x : numpy.array
        Vector de 80 dimensiones representando todas las posiciones de los 20 eslabones
    
    Resultado
    ---------
    rv : float
        Energía potencial total de la cadena según la configuración dada en `x`.

    """
    y_coords = x[1::2]  # energia potencial se calcula solo con las coordenadas y
    rv = 0.5 * g * sum(y_coords)
    return rv

def link_size(x, l):
    """Objective function to minimize.
    
    Parámetros
    ----------
    x : numpy.array
        Vector de 80 dimensiones representando las posiciones de los 20 eslabones
    l : int
        Indice del eslabón, debe ser uno de 0, 1, 2, ..., 19
    
    Resultado
    ---------
    rv : float
        Length of the link `l` given by the configuration in `x`.
    
    """
    i = 4*l
    rv = (x[i] - x[i+2])**2 + (x[i+1] - x[i+3])**2
    return round(rv, 3)

A continuación, defino las restricciones $h_i$. Empezamos por las que fijan el primer y último eslabón;

In [148]:
cons = [
    {'type': 'eq', 'fun': lambda x: x[0]},
    {'type': 'eq', 'fun': lambda x: x[1]},
    {'type': 'eq', 'fun': lambda x: x[79]},
    {'type': 'eq', 'fun': lambda x: x[78] - 16}
]

Seguimos con las que imponen que los extremos de los eslabones consecutivos se toquen;

In [149]:
for k in range(1, 20):
    i = 4*k
    j = 4*k + 1
    cons.append({'type': 'eq', 'fun': lambda x, i=i: x[j] - x[j-2]})
    cons.append({'type': 'eq', 'fun': lambda x, i=i: x[j] - x[j-2]})

Por último le imponemos a los eslabones que mantengan su tamaño.

In [150]:
for i in range(20):
    # 20 restricciones que fijan el tamaño de los eslabones
    cons.append({'type': 'ineq', 'fun': lambda x, i=i: 1 - link_size(x, i)})

# cons.append({'type': 'eq', 'fun': lambda x: sum(x[2::4] - x[0::4]) - 16})
print(f'Total: {len(cons)} restricciones.')

Total: 62 restricciones.


Con las restricciones definidas puedo usarlas puedo armar la solución inicial y chequear su factibilidad. Notar que es la misma solución inicial que propuse en (a) pero descripta con cuatro coordenadas para cada eslabón acorde al modelo que elegí.

In [151]:
x0 = np.array([  # start point: each row is a link (xl, yl, xr, yr)
    0., 0., 0.8, 0.6,
    0.8, 0.6, 1.6, 1.2,
    1.6, 1.2, 2.4, 1.8,
    2.4, 1.8, 3.2, 2.4,
    3.2, 2.4, 4., 3.,
    4., 3., 4.8, 3.6,
    4.8, 3.6, 5.6, 4.2,
    5.6, 4.2, 6.4, 4.8,
    6.4, 4.8, 7.2, 5.4,
    7.2, 5.4, 8., 6.,
    8., 6., 8.8, 5.4, 
    8.8, 5.4, 9.6, 4.8,
    9.6, 4.8, 10.4, 4.2,
    10.4, 4.2, 11.2, 3.6,
    11.2, 3.6, 12., 3.,
    12., 3., 12.8, 2.4,
    12.8, 2.4, 13.6, 1.8,
    13.6, 1.8, 14.4, 1.2,
    14.4, 1.2, 15.2, 0.6,
    15.2, 0.6, 16, 0
])
if (np.array([h['fun'](x0) for h in cons]) == 0).all():
    print(f"Factibilidad: =)")
else:
    print((np.array([h['fun'](x0) for h in cons]) == 0).all())

Factibilidad: =)


Con todo esto finalmente estamos en condiciones de pasarle nuestras especificaciones al solver. En este caso uso la rutina `minimize` de `scipy.optimize`. [Ver doumentación.](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.minimize.html)

In [145]:
from scipy.optimize import minimize
solution = minimize(obj_function, x0, constraints=cons)
solution.success, solution.status, solution.message

(False, 6, 'Singular matrix C in LSQ subproblem')

In [146]:
x_sol = solution.x[0::2]
y_sol = solution.x[1::2]
fig = go.Figure()
fig.add_trace(go.Scatter(x=x_sol, y=y_sol, mode='markers'))
fig.update_yaxes(range=[6.5, -0.5])
fig.show()